In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torchvision.models import vit_b_16

In [2]:
transform_train=transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5,0.5,0.5],
        std=[0.5,0.5,0.5]
    )
])

In [3]:
transform_test=transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5,0.5,0.5],
        std=[0.5,0.5,0.5]
    )
])

In [4]:
train_data=datasets.CIFAR10(
    root='./data',train=True,download=True,transform=transform_train
)

test_data=datasets.CIFAR10(
    root='./data',train=False,download=True,transform=transform_test
)

100%|██████████| 170M/170M [00:12<00:00, 13.8MB/s] 


In [5]:

train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=64, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=64, shuffle=False
)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = vit_b_16(weights="IMAGENET1K_V1")

# Replace classification head
model.heads.head = nn.Linear(model.heads.head.in_features, 10)

model = model.to(device)


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:01<00:00, 227MB/s] 


In [7]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.AdamW(model.parameters(),lr=3e-4,weight_decay=1e-4)

In [ ]:
epochs=2
for epoch in range(epochs):
    model.train()
    correct=0
    total=0
    train_loss=0
    for images,labels in train_loader:
        images,labels=images.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs=model(images)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()


        train_loss +=loss.item()
        _,predicted=outputs.max(1)
        total +=labels.size(0)
        correct +=predicted.eq(labels).sum().item()


    print(f"Epoch [{epoch}/10"
         f"Train Loss: {train_loss:.4f}"
        )

## device

In [ ]:
print(torch.cuda.is_available())
print(device)
